In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=74e459efcd6dcf8e988e4be3890ac168e3838ed77c18b62a3d1feede550693ca
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.6MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F9 '''
def F9(X):
    f = bn.F9()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 395.18 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_9_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_9_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(0,10):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=958198, Tue Apr 14 16:37:05 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 4.031899351307223e+07 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 4.009578957978714e+07 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [40664864.04403681] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [40494181.74093337] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 3.997634957881572e+07 1.0e+00 2.34e+00  2e+00  2e+00 0:00.4
   27    513 3.841442940886762e+07 1.1e+00 2.03e+00  2e+00  2e+00 0:03.5
   58   1102 3.828902032766036e+07 1.1e+00 1.91e+00  2e+00  2e+00 0:07.5
   97   1843 3.659178320546345e+07 1.2e+00 1.61e+00  2e+00  2e+00 0:12.5
  100   1900 3.637297316080497e+07 1.2e+00 1.58e+00  2e+00  2e+00 0:12.9
  155   2945 2.486775304200704e+07 1.2e+00 1.01e+00  1e+00  1e+00 0:20.0
  200   3800 1.498568805194678e+07 1.3e+00 6.47e-01  6e-01  7e-01 0:25.7
  272   5168 1.620391453019604e+06 1.3e+00 3.02e-01  3e-01  3e-01 0:34.8
  300   5700 -2.939332896644771e+05 1.3e+00 2.23e-01  2e-01  2e-01 0:38.4
  387   7353 -2.308767649316147e+06 1.3e+00 8.26e-02  8e-02  8e-02 0:49.4
  400   7600 -2.355905905500449e+06 1.3e+00 7.23e-02  7e-02  7e-02 0:51.1
  500   9500 -2.567860492984295e+06 1.3e+00 2.35e-02  2e-02  2e-02 1:03.9
  600  11400 -2.589526767452158e+06 1.3e+00 7.48e-03  7e-03  8e-03 1:16.5
  700  13300 -2.592291604703620e+06 1.3e+00 2.

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [40597407.50487186] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=40597407.504872, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=40597407.504872, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

termination on tolfun=1e-11 (Tue Apr 14 17:01:15 2020)
final/bestever f-value = 4.059741e+07 4.059741e+07
incumbent solution: [ 1.42571962  0.57366544  0.86337714 -2.31482875  2.88984282 -2.63054831
 -4.97536612  1.72724299 ...]
std deviations: [2.43486736 2.43570215 2.43525754 2.43508793 2.43544342 2.43542067
 2.43532085 2.43531286 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=988590, Tue Apr 14 17:01:15 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 4.545646463833039e+07 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 4.189147668294293e+07 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


...

   26    494 3.771682658585133e+07 1.1e+00 1.98e+00  2e+00  2e+00 0:03.5
   57   1083 3.640848660480645e+07 1.1e+00 1.70e+00  2e+00  2e+00 0:07.6
   93   1767 3.286716725962702e+07 1.2e+00 1.40e+00  1e+00  1e+00 0:12.6
  100   1900 3.346763524773490e+07 1.2e+00 1.32e+00  1e+00  1e+00 0:13.6
  152   2888 2.329494403983038e+07 1.2e+00 8.43e-01  8e-01  9e-01 0:20.7
  200   3800 1.028207929870050e+07 1.2e+00 5.30e-01  5e-01  5e-01 0:27.2
  265   5035 7.810998186802119e+05 1.2e+00 2.59e-01  3e-01  3e-01 0:36.2
  300   5700 -1.158957136156902e+06 1.2e+00 1.80e-01  2e-01  2e-01 0:40.9
  381   7239 -2.268707181159824e+06 1.2e+00 7.55e-02  7e-02  8e-02 0:51.9
  400   7600 -2.387122915049694e+06 1.2e+00 6.22e-02  6e-02  6e-02 0:54.5
  497   9443 -2.569033814227484e+06 1.2e+00 2.16e-02  2e-02  2e-02 1:07.5
  500   9500 -2.573453938449718e+06 1.2e+00 2.09e-02  2e-02  2e-02 1:07.9
  600  11400 -2.590473297347605e+06 1.2e+00 6.93e-03  7e-03  7e-03 1:21.3
  700  13300 -2.592403963974625e+06 1.2e+00 2

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [40597407.50487186] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=40597407.504872, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=40597407.504872, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

termination on tolfun=1e-11 (Tue Apr 14 23:56:37 2020)
final/bestever f-value = 4.059741e+07 4.059741e+07
incumbent solution: [-0.09908571  1.05698873  4.91667838  1.42196597 -2.24286616  3.58537562
 -0.85414815 -2.40679252 ...]
std deviations: [2.43712108 2.4374379  2.4373665  2.43690782 2.43736254 2.43746
 2.43702355 2.43747894 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=1072898, Tue Apr 14 23:56:37 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 4.316064068371096e+07 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 3.823444708187386e+07 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [50035170.45270903] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [40745559.31829384] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 3.670449157168384e+07 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   31    589 1.302568537554517e+07 1.1e+00 1.88e+00  2e+00  2e+00 0:03.4
   68   1292 1.222388450219836e+06 1.1e+00 1.56e+00  2e+00  2e+00 0:07.6
  100   1900 -4.359798519397113e+06 1.2e+00 1.33e+00  1e+00  1e+00 0:11.1
  155   2945 -1.788342029248475e+07 1.2e+00 8.80e-01  9e-01  9e-01 0:17.2
  200   3800 -2.672635382165577e+07 1.2e+00 6.14e-01  6e-01  6e-01 0:22.2
  272   5168 -3.313642327400158e+07 1.2e+00 3.56e-01  4e-01  4e-01 0:30.2
  300   5700 -3.376838673323414e+07 1.2e+00 2.81e-01  3e-01  3e-01 0:33.3
  391   7429 -3.577099233298146e+07 1.2e+00 1.43e-01  1e-01  1e-01 0:43.4
  400   7600 -3.593429074670060e+07 1.3e+00 1.37e-01  1e-01  1e-01 0:44.4
  500   9500 -3.659406952741572e+07 1.3e+00 8.19e-02  8e-02  8e-02 0:55.4
  600  11400 -3.684515933741501e+07 1.5e+00 5.66e-02  5e-02  6e-02 1:06.6
  700  13300 -3.698497448796973e+07 1.8e+00 4.32e-02  4e-02  5e-02 1:17.7
  800  15200 -3.708510073415145e+07 2.1e+

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F9_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F9(X_Values.iloc[i,:200])
    SVM_Fun [i] = F9(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F9(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([94775.49122863]), count=array([1])) 0.002142556518660951
SVM
ModeResult(mode=array([38651644.70460992]), count=array([1])) 6134647.50385713
ELN
ModeResult(mode=array([39877105.23937325]), count=array([1])) 10345998.255703753
